In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *

In [3]:
from fastai.text.all import *

# **Get Dataset**

In [4]:
path = untar_data(URLs.IMDB)

NOTE: `get_text_files` gets all the text files in a path. We can also optionally pass `folders` to restrict the search to a particular list of subfolders:

In [5]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/README')]

In [5]:
files = get_text_files(path, folders = ["train", "test", "unsup"])

# Create a Language Model for IMDB

In [6]:
get_imdb = partial(get_text_files, folders = ['train', 'test', 'unsup'])

In [7]:
dls_lm = DataBlock(
    blocks = TextBlock.from_folder(path, is_lm=True),
    get_items = get_imdb, splitter = RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [40]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj i 've just revisited this fondly remembered bit of cinematic madness from my early days , and must urge you to beg steal or borrow it . \n\n xxmaj the story begins with a duel between a righteous xxmaj shaolin priest and our villain xxmaj abbot xxmaj white , needless to say , xxmaj abbot xxmaj white kicks xxmaj buddhist ass , and wages his campaign against xxmaj shaolin unhindered with the aid of his new ninja allies","xxmaj i 've just revisited this fondly remembered bit of cinematic madness from my early days , and must urge you to beg steal or borrow it . \n\n xxmaj the story begins with a duel between a righteous xxmaj shaolin priest and our villain xxmaj abbot xxmaj white , needless to say , xxmaj abbot xxmaj white kicks xxmaj buddhist ass , and wages his campaign against xxmaj shaolin unhindered with the aid of his new ninja allies ("
1,"xxmaj after a few times , i get bored and changed the channel . i still love xxmaj robert xxmaj culp and xxmaj patricia xxmaj crowley and xxmaj ray xxmaj milland in their roles but the story was weaker in this episode than in the others . xxmaj first , xxmaj robert xxmaj culp plays an investigator for xxmaj ray xxmaj milland 's character . xxmaj he hires him to investigate his young pretty wife played by xxmaj patricia xxmaj","after a few times , i get bored and changed the channel . i still love xxmaj robert xxmaj culp and xxmaj patricia xxmaj crowley and xxmaj ray xxmaj milland in their roles but the story was weaker in this episode than in the others . xxmaj first , xxmaj robert xxmaj culp plays an investigator for xxmaj ray xxmaj milland 's character . xxmaj he hires him to investigate his young pretty wife played by xxmaj patricia xxmaj crowley"


**Now that our data is ready, we can fine-tune the pretrained language model**

## Fine-Tuning the Language Model

In [8]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics = [accuracy, Perplexity()]
).to_fp16()

In [9]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.004746,3.900990,0.300720,49.451393,34:16


**Save the Model state**

In [43]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

**Load the Model**

In [11]:
# learn = learn.load('1epoch')

**Continue Training**

In [10]:
learn.unfreeze()

In [11]:
learn.fit_one_cycle(5,2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.766863,3.752828,0.317781,42.641506,36:19
1,3.669888,3.663725,0.327585,39.006378,36:12
2,3.533029,3.604167,0.335046,36.751049,36:16
3,3.452684,3.572528,0.339135,35.606495,36:13
4,3.371913,3.572680,0.339655,35.611897,36:14


In [12]:
learn.export('/notebooks/FASTAI_2022/review_generator.pkl')

# **Text Generation**

In [14]:
TEXT = "I like this movie because"
N_WORDS = 50
N_SENTENCES = 2
predicts = [learn.predict(TEXT, N_WORDS, temperature = 0.75) for _ in range(N_SENTENCES)]

In [15]:
print("\n".join(predicts))

i like this movie because it has the ability to make you feel clever and clever . The way the characters are portrayed is clever and i can not help but feel they are in one movie . i also like that this movie is also a movie that makes you feel good .
i like this movie because it has a little bit of a twist , but it 's a very good movie . I 'm a big fan of the Jay Leno & Letterman show . i am really impressed . i watch every show on TV & the networks keep
